## Cамостоятельная реализация решающего дерева  

#### Определение дерева

Решающее дерево - **бинарное** дерево, в котором:
1. Каждой *внутренней вершине* $v$ присвоен предикат предсказания: $B_v : \mathbb{X} \rightarrow \{0, 1\}$
2. Каждой *листовой вершине* $v$ присвоен прогноз $C_v: \mathbb{Y}$, где $\mathbb{Y}$ - область значений таргета

Каждый проход дерева начинается из корня. При прохождении очередной вершины мы двигаемся: *вправо*, если $B_v(x) = 1$; *влево*, если $B_v(x) = 0$. 

При достижении листа на объекте $x$, прогнозом для него будет являться $C_v$

Особенности решающего дерева:
1. Полученная функция кусочно-постоянная $\rightarrow$ **не получится применить градиентные методы**
2. Дерево **не может экстраполировать данные** за пределы уже имеющейся области значений признаков обучающей выборки
3. Дереву свойственно **переобучение**

#### Решающий пень

Дерево можно разбить на составляющие - решающие пни. Они будут представлять собой одну и вершину и два дочерних листа.

Вершину мы будем разделять на листы при помощи предиката $B_{j, t}(x_i)$ . Качество разбиения мы будем оценивать при помощи критерия ветвления $Branch$.

На листьях подзнее мы можем принять решение о необходимости дальнейшего разбиения -> построения еще одного решающего пня.


#### Сложность решающего пня

Пусть у нас есть матрица значений признаков $X \in \mathbb{R}^{D \times N}$ и вектор таргетов $Y \in \mathbb{R}^N$.

В основе вершины пня будет находится разделяющий предикат:
$$B_{j, t}(x_i) = \mathbb{I}\left[x_{ij} \le t\right]$$
Мы будем проходить не по самим значениям признаков, а по средним между значениями.
$$x_i < t_i \le x_{i+1}$$
Поэтому мы пройдем всего по $N-1$ значению каждого признака.

Тогда решение на пне примет вид:
$$(j_{opt}, t_{opt}) = \arg\min_{j,t} L \left( B_{j, t}, X, y \right)$$

Для того чтобы рассчитать $loss$, необходимо еще одного прохождение по $N$, в результате получим, что полный алгоритм решающего пня будет выполняться за $O(DN^2)$, где $D$ - кол-во признаков, $N$ - кол-во объектов.

#### Главная проблема решающих деревьев

Запустив предложенный выше алгоритм рещающего пня рекурсивно, он будет выполняться до тех пор, пока полностью не выучит обучающую выборку -> переобучится.

Если мы поставим задачу найти оптимальное решающее дерево при минимальном количестве разбиений, то решение такой задачи не сможем найти за полиномиальное время, т. к. она относится к np - полным задачам.

Чтобы решить ситуацию в настоящий момент пользуются двумя способами:
1. Жадный алгоритм
2. Оптимизация исходного алгоритма ассимптотически и в константу раз

#### Жадный алгоритм *построения* решающего дерева

У нас уже есть матрица значений признаков $X$, определенная выше. Пусть $X_m \subset X$ - множество всех объектов попавших в текущий лист.

1. Создаем вершину $v$
2. **Если**: выполнен ли критерий остановки $Stop(X_m)$, **то** останавливаемся и ставим ответ $Answ(X_m)$, объявив вершину листом.
3. **Иначе**: Находим предикат $B_{j, t}$ имеющий лучшее разбиение на листы $X_m \rightarrow X_l, X_r$. Максимизируя критерий ветвления $Branch(X_m)$
4. Рекурсивно выполняем алгоритм для листьев $X_l, X_r$

Подробнее разберем каждую функцию представленную в алгоритме:
1. $Stop(X_m)$ - критерий остановки. Необходим для того, чтобы при построении решающего дерева мы могли остановиться и не переобучиться.
2. $Answ(X_m)$ - функция вычисляющая ответ для листа, по попавшим в него объектам $X_m$. Может быть:
   - Для задачи *классификации* ответ может быть: ***меткой самого частого класса*** или ***оценкой*** дискрет. ***распределения вероятностей классов*** для объектов в листе.
   - Для задачи *регрессии*: ***средним, медианой или любой другой статистикой***
   - Для любой задачи *простой моделью*: ***линейной функцией, синусойдой или любой другой функцией***
3. $Stop(X_m)$ - критерий остановки при достижении определенных параметров дерево (про регуляризацию деревьев ниже)
4. $Branch(X_m, j, t)$ - критерий ветвления

##### Подробнее про критерий ветвления

Ответы дерева можем закодировать как: 
- $\bar{c} \in \mathbb{R}^k$ для регрессии и классификации
- $\bar{c} = (c_1, ..., c_k) \in \mathbb{R}: \sum_i \bar{c} = 1$ для дискретного распределения вероятностей классов

Предположим что задана функция потерь $L(y_i, c)$. *В момент поиска оптимального разделения* $X_m = X_l \cup X_r$, мы можем *вычислить* для $X_m$ *константный таргет* $c$ (предикт дерева, если бы вершина была терминальной) и связанный с ним значение ф-ии потерь $L$. А именно - константа $c$ должна минимизировать среднее качество $L$.
$$\frac{1}{|X_m|} \sum_{(x_i, y_i) \in X_m} L(y_i, c)$$

Тогда оптимальное значение:
$$H(X_m) = \min_{c \in Y} \frac{1}{|X_m|} \sum_{(x_i, y_i) \in X_m} L(y_i, c)$$

$H(X_m)$ - называется ***неоднородностью (impurity)***, чем она ниже, тем предикт дерева ближе к *некоторому константному значению*.

Таким же образом можно вычислить информативность всего решающего пня:
1. $X_l$ - объекты попавшие в левый лист
2. $X_r$ - объекты попавшие в правый лист
3. $c_l$, $c_r$ - константы предсказаний в каждом листе для определенного $B_{j, t}(X_m)$

Фукнция потерь всего пня:
$$\frac{1}{|X_m|} \left( \sum_{(x_i, y_i) \in X_l} L(y_i, c_l) + \sum_{(x_i, y_i) \in X_r} L(y_i, c_r) \right)$$

Связать это с информативностью:
$$ \frac{1}{|X_m|} \left( \frac{|X_l|}{|X_l|} \sum_{(x_i, y_i) \in X_l} L(y_i, c_l) + \frac{|X_r|}{|X_r|} \sum_{(x_i, y_i) \in X_r} L(y_i, c_r) \right) = $$
$$ = \frac{|X_l|}{|X_m|} H(X_l) + \frac{|X_r|}{|X_m|} H(X_r) $$

Чтобы получить качество разбиения мы найдем разницу информативностей вершины и получившихся листьев:
$$Branch(X_m, j, t) = |X_m| * H(X_m) - |X_l| * H(X_l) - |X_r| * H(X_r)$$


Функция потерь выбирается под конкретную задачу, подробнее про выбранные функции будет при их реализации

#### Регуляризация решающих деревьев

Так как дерево обязательно переобучится, если его не ограничить, то необходимо упомянуть о методах регуляризации деревьев:
1. Ограничения по максимальной глубине
2. Ограничения на минимальное количество объектов в листе
3. Ограничение на максимальное количество листьев в дереве
4. Требование, чтобы функционал качества при делении текущей подвыборки на две улучшался не менее чем на $s\%$

Перечисленные действия возможно выполнить на разных этапах действия алгоритма:
- Pre-pruning: В процессе построения дерева при достижении критерия остановки
- Post-pruning: После построения дерева, удалить некоторые вершины так, чтобы качество упало не сильно. Проверяя качество на val

#### Функции потерь: классификация

##### Gini (Джини)

Пусть предсказание модели - распределение вероятности классов: $\bar{c} = (c_1, ..., c_k); \sum_{i=1}^k c_i = 1$

Тогда посчитаем *Brier score* на вероятностях классов при получившемся разбиении.
$$BS(c) = \frac{1}{N} \sum_{i=1}^k \left( c_k - \mathbb{I} \left[ y_i = k \right] \right)^2$$

BS - при идеальном предсказании имеет значение. Чем меньше разница между предсказанным классом и вероятностью, тем ниже метрика.

Следовательно функция неоднородности будет иметь вид минимизации функционала по каждому из классу

$$H(X_m)= \min_{\sum_k c_k = 1} \frac{1}{X_m} \sum_{(x_i, y_i) \in X_m} \sum_{k=1}^K \left( c_k - \mathbb{I} \left[ y_i = k \right] \right)^2$$

Логично, что наименьшее зачение метрики достигается на $c$ состоящем из выборочных оценок частот классов в подвыборке $X_m$: $(p_1, ..., p_k), p_i = \frac{1}{|X_m|} \sum_i \mathbb{I} \left[ y_i = k \right]$.

Если подставить вектор выборочых оценок частот классов в форулу неоднородности, то получится свести задачу к следующему виду:
$$H(X_m) = \sum_{k=1}^K p_k (1 - p_k)$$

Критерий Джини допускает следующую интерпретацию:

$H(X_m)$ равно математическому ожиданию числа неправильных ответов, если мы будем присваивать им случайные метки классов согласно дискретному распределению заданному $(p_1, ..., p_k)$

#### Реализация класса обучения

In [1]:
## TODO: rewrite fit method with BFS APPROACH
## TODO: SORT FEATURES ONE TIME before fit
## TODO: find split with VECTORIZED CUMSUM
## TODO: predict with OPTIMIZED BATCH

In [2]:
import numpy as np

In [3]:
import torch
import torch.nn.functional as F
from math import log2, floor

class SelfDecisionTreeClassifier():
    '''
    Decision Tree Classifier
    
    Criterion: Gini, LogLoss, Entropy

    Regularization presented with:
    - Max depth of tree
    - Min_leaf_samples of tree
    
    Tree is building with DFS approach (# To optimize can use BFS)
    The decisive stump predicate is loking best split on all features all values
    So to partially optimize calculations we will use PyTorch and GPU
    '''
    def __init__(self, device: str, criterion: str = 'gini', max_depth: int | None = None, min_leaf_samples: int | None = 1):
        assert criterion in ('gini', 'entropy', 'log')

        self.device = device
        
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_leaf_samples = min_leaf_samples
        self.K = None
        self.is_fitted = False
        
        self.__thresholds = []
        self.__features = []
        self.__impurity_list = []
        self.__is_leaf = []
        self.__split = []
        self.__probs = []

        self.__all_lists = [
            self.__thresholds,
            self.__features,
            self.__impurity_list,
            self.__is_leaf,
            self.__split,
            self.__probs
        ]
        
        self.N = 0
        self.D = 0

    def __get_childerns_idxs(self, cur_idx: int) -> tuple[int, int]:
        return (2 * cur_idx + 1, 2 * cur_idx + 2)
    
    def __extend_lists(self, cur_idx: int) -> None:
        _, r_idx = self.__get_childerns_idxs(cur_idx)
        
        for i in range(len(self.__all_lists)):
            if len(self.__all_lists[i]) < r_idx:
                self.__all_lists[i].extend([None] * (r_idx - len(self.__all_lists[i])))
            
    def __stop(self, cur_idx: int) -> bool:
        if self.max_depth is not None:
            if floor(log2(cur_idx + 1)) >= self.max_depth:
                return True
                
        if self.min_leaf_samples is not None:
            if sum(self.__split[cur_idx]) <= self.min_leaf_samples:
                return True
        else:
            return False

    def __calc_probs(self, indices: torch.LongTensor | torch.Tensor) -> torch.Tensor:
        return torch.bincount(self.y[indices].flatten(), minlength=self.K) / indices.shape[0]
    
    def _gini_loss(self, indices: torch.LongTensor) -> float:
        probs = self.__calc_probs(indices)
        return torch.sum(probs * (1 - probs))

    def _log_loss(self, indices: torch.LongTensor) -> float:
        pass
    
    def _entropy_loss(self, indices: torch.LongTensor) -> float:
        pass
        
    def __impurity(self, indices: torch.LongTensor) -> float:
        loss_function = getattr(self, f'_{self.criterion}_loss')
        return loss_function(indices)

    def __create_leaf(self, cur_idx: int) -> None:
        self.__is_leaf[cur_idx] = True
        self.__probs[cur_idx] = self.__calc_probs(torch.where(self.__split[cur_idx])[0]).cpu().numpy()

        # print('---' * 4, ' LEAF ', '---' * 4,)
        # print('Now at index: ', cur_idx)
        # print('Obj count: ', sum(self.__split[cur_idx]))
        # print('Probs: ', self.__probs[cur_idx])
        # print('---' * 11, end='\n\n')
        
        return 
        
    ## Recursive
    def __split_node(self, cur_idx: int) -> None:
        if self.__stop(cur_idx):
            self.__create_leaf(cur_idx)
            return
        
        best_branch_score = -float('inf') # Dict for all?
        best_feature_idx = None
        best_threshold = None
        best_Xr_idxs = None
        best_Xl_idxs = None
        best_Hr = None
        best_Hl = None
        flag = False

        len_Xm = sum(self.__split[cur_idx])
        Hm = self.__impurity_list[cur_idx] 
        
        for feature_idx in range(self.D):
            sorted_vals, indices = torch.sort(self.X[self.__split[cur_idx], feature_idx])
            
            prev_val = None       
                 
            for idx in range(0, indices.shape[0] - 1):
                
                cur_val = sorted_vals[idx]
                if cur_val == prev_val:
                    continue
                
                threshold = (self.X[indices[idx], feature_idx] + self.X[indices[idx + 1], feature_idx]) / 2
                
                Xr_idxs = indices[:idx + 1]
                Xl_idxs = indices[idx + 1:]

                Hr = self.__impurity(Xr_idxs) # Hr = H(Xr)
                Hl = self.__impurity(Xl_idxs) # Hl = H(Xl)

                branch_score = len_Xm * Hm - Xl_idxs.shape[0] * Hl - Xr_idxs.shape[0] * Hr

                if branch_score > best_branch_score:
                    best_branch_score = branch_score
                    best_feature_idx = feature_idx
                    best_threshold = threshold
                    best_Xr_idxs = Xr_idxs
                    best_Xl_idxs = Xl_idxs
                    best_Hr = Hr
                    best_Hl = Hl                   
                    flag = True
                
                prev_val = cur_val
        
        if not(flag):
            self.__create_leaf(cur_idx)
            return

        # print('NODE')
        # print('Now at index: ', cur_idx)
        # print('Main node of stump objects count: ', sum(self.__split[cur_idx]))
        # print('Feature to split by: ', best_feature_idx)
        # print('Best founded threshold: ', best_threshold.item())
        # print('Best branch score: ', best_branch_score.item())
        # print('Element count in r_leaf, l_leaf: ', best_Xl_idxs.shape[0], best_Xr_idxs.shape[0], end='\n\n')

        ## заполнить массивы
        self.__extend_lists(cur_idx)
        self.__thresholds[cur_idx] = best_threshold
        self.__features[cur_idx] = best_feature_idx
        self.__is_leaf[cur_idx] = False
        
        r_idx, l_idx = self.__get_childerns_idxs(cur_idx)
        self.__extend_lists(r_idx)
        self.__extend_lists(l_idx)
        
        self.__impurity_list[r_idx] = best_Hr 
        mask_r = torch.zeros(self.N, dtype=torch.bool, device=self.device)
        mask_r[best_Xr_idxs] = True
        self.__split[r_idx] = mask_r

        self.__impurity_list[l_idx] = best_Hl 
        mask_l = torch.zeros(self.N, dtype=torch.bool, device=self.device)
        mask_l[best_Xl_idxs] = True
        self.__split[l_idx] = mask_l

        ## вызвать рекурсивно на листах
        self.__split_node(r_idx)
        self.__split_node(l_idx)         

    
    def __predict_single(self, X: torch.tensor) -> float:
        idx = 0
        while not(self.__is_leaf[idx]):
            feat = self.__features[idx]
            thr = self.__thresholds[idx]
            if X[feat] >= thr:
                idx = 2 * idx + 2
            else:
                idx = 2 * idx + 1
        return self.__probs[idx]
        
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        assert X.shape[0] == y.shape[0], 'N dimension of X and y must be equal'
        
        self.X = torch.tensor(X, device=self.device)
        self.y = torch.tensor(y, device=self.device)
        
        self.N = X.shape[0]
        self.D = X.shape[1]

        self.K = torch.unique(self.y).shape[0]
        
        self.__extend_lists(0)
        self.__impurity_list[0] = self.__impurity(torch.nonzero(torch.ones_like(self.X[:, 0], device=self.device)))
        self.__split[0] = torch.ones_like(self.y, dtype=bool, device=self.device)
        
        self.__split_node(0)
        self.is_fitted = True
        
        return
        
    def predict(self, X: np.ndarray) -> list[torch.Tensor]:
        assert self.is_fitted == True, 'You must fit before predict'
        assert X.shape[1] == self.X.shape[1], 'Number of features must be equal fitted number of features'
        
        X = torch.tensor(X, dtype=float, device=self.device)
        probs = []
        for i in range(X.shape[0]):
            probs.append(self.__predict_single(X[i, :]))
        return probs
        
        

In [21]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)
X = np.random.rand(100, 6)
y = np.random.randint(0, 2, 100)
# print(np.sort(random_array, axis=0))

cpu


In [22]:
sk_classifier = DecisionTreeClassifier(max_depth=4, min_samples_leaf=2)
sk_classifier.fit(X, y)
pred_proba_sk = sk_classifier.predict_proba(X)
sk_pred_labels = [0 if arr[0] > 0.5 else 1 for arr in pred_proba_sk]

In [23]:
# random_array = np.random.rand(10, 6)
# print(np.sort(random_array, axis=0))
self_classifier = SelfDecisionTreeClassifier(device, max_depth=4, min_leaf_samples=2)
self_classifier.fit(X, y)
pred_probs_self = np.array(classifier.predict(X))
self_pred_labels = [0 if arr[0] > 0.5 else 1 for arr in pred_probs_self]

In [24]:
from sklearn.metrics import f1_score

print('Self tree f1: ', f1_score(y, self_pred_labels))
print('Sklearn tree f1: ', f1_score(y, sk_pred_labels))

Self tree f1:  0.7555555555555555
Sklearn tree f1:  0.835820895522388


### Использование на данных

In [38]:
from sklearn.model_selection import train_test_split

In [48]:
data = np.load('./data/train.npy')
X, y = data[:, 1:], data[:, :1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [49]:
print(X_train.shape, y_train.shape)

(711, 9) (711, 1)


In [50]:
print(X_test.shape, y_test.shape)

(178, 9) (178, 1)


In [51]:
from sklearn.tree import DecisionTreeClassifier

In [54]:
sk_tree = DecisionTreeClassifier(max_depth=5, min_samples_leaf=5)
sk_tree.fit(X_train, y_train)
pred_proba_sk = sk_tree.predict_proba(X_test)
sk_pred_labels = [0 if arr[0] > 0.5 else 1 for arr in pred_proba_sk]
f1_score(sk_pred_labels, y_test)

0.7419354838709677

In [56]:
self_tree = SelfDecisionTreeClassifier(device, max_depth=5, min_leaf_samples=5)
self_tree.fit(X_train, y_train)
pred_probs_self = np.array(self_tree.predict(X_test))
self_pred_labels = [0 if arr[0] > 0.5 else 1 for arr in pred_probs_self]

RuntimeError: "bincount_cpu" not implemented for 'Double'